# Quickstart

Tutorial for [NautilusTrader](https://nautilustrader.io/docs/) a high-performance algorithmic trading platform and event driven backtester.

[View source on GitHub](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/quickstart.ipynb).

## Overview

This quickstart tutorial shows you how to get up and running with NautilusTrader backtesting using FX data.
To support this, we provide pre-loaded test data in the standard Nautilus persistence format (Parquet).


## Prerequisites
- Python 3.12+ installed.
- [NautilusTrader](https://pypi.org/project/nautilus_trader/) latest release installed (`uv pip install nautilus_trader`).
- [JupyterLab](https://jupyter.org/) or similar installed (`uv pip install jupyterlab`).

## 1. Get sample data

To save time, we have prepared sample data in the Nautilus format for use with this example. 
Run the next cell to download and set up the data (this should take ~ 1-2 mins).

For further details on how to load data into Nautilus, see [Loading External Data](https://nautilustrader.io/docs/latest/concepts/data#loading-data) guide.

In [34]:
import os
import urllib.request
from pathlib import Path

from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider


# Change to project root directory
original_cwd = os.getcwd()
project_root = os.path.abspath(os.path.join(os.getcwd(),".."))
os.chdir(project_root)

print(f"Working directory: {os.getcwd()}")

# Create catalog directory
catalog_path = Path("catalog")
catalog_path.mkdir(exist_ok=True)

print(f"Catalog directory: {catalog_path.absolute()}")

try:
    # Download EUR/USD sample data
    print("Downloading EUR/USD sample data...")
    url = "https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/raw_data/fx_hist_data/DAT_ASCII_EURUSD_T_202001.csv.gz"
    filename = "EURUSD_202001.csv.gz"

    print(f"Downloading from: {url}")
    urllib.request.urlretrieve(url, filename)  # noqa: S310
    print("Download complete")

    # Create the instrument using the current schema (includes multiplier)
    print("Creating EUR/USD instrument...")
    instrument = TestInstrumentProvider.default_fx_ccy("EUR/USD")

    # Load and process the tick data
    print("Loading tick data...")
    wrangler = QuoteTickDataWrangler(instrument)

    df = CSVTickDataLoader.load(
        filename,
        index_col=0,
        datetime_format="%Y%m%d %H%M%S%f",
    )
    df.columns = ["bid_price", "ask_price", "size"]
    print(f"Loaded {len(df)} ticks")

    # Process ticks
    print("Processing ticks...")
    ticks = wrangler.process(df)

    # Write to catalog
    print("Writing data to catalog...")
    catalog = ParquetDataCatalog(str(catalog_path))

    # Write instrument first
    catalog.write_data([instrument])
    print("Instrument written to catalog")

    # Write tick data
    catalog.write_data(ticks)
    print("Tick data written to catalog")

    # Verify what was written
    print("\nVerifying catalog contents...")
    test_catalog = ParquetDataCatalog(str(catalog_path))
    loaded_instruments = test_catalog.instruments()
    print(f"Instruments in catalog: {[str(i.id) for i in loaded_instruments]}")

    # Clean up downloaded file
    os.unlink(filename)
    print("\nData setup complete!")

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    os.chdir(original_cwd)
    print(f"Changed back to: {os.getcwd()}")

Working directory: C:\Users\saill\Desktop\nautilus
Catalog directory: C:\Users\saill\Desktop\nautilus\catalog
Download complete
Creating EUR/USD instrument...
Loading tick data...


c:\Users\saill\Desktop\nautilus\.venv\Lib\site-packages\nautilus_trader\persistence\loaders.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


Loaded 1637768 ticks
Processing ticks...
Writing data to catalog...
Instrument written to catalog
Tick data written to catalog

Verifying catalog contents...
Instruments in catalog: ['EUR/USD.SIM']

Data setup complete!
Changed back to: C:\Users\saill\Desktop\nautilus\notebooks


In [3]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog

## 2. Set up a Parquet data catalog

If everything worked correctly, you should be able to see a single EUR/USD instrument in the catalog.

In [18]:
# Load the catalog from the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
catalog_path = os.path.join(project_root, "catalog")

catalog = ParquetDataCatalog(catalog_path)
instruments = catalog.instruments()

print(f"Loaded catalog from: {catalog_path}")
print(f"Available instruments: {[str(i.id) for i in instruments]}")

if instruments:
    print(f"\nUsing instrument: {instruments[0].id}")
else:
    print("\nNo instruments found. Please run the data download cell first.")

Loaded catalog from: C:\Users\saill\Desktop\catalog
Available instruments: ['EUR/USD.SIM']

Using instrument: EUR/USD.SIM


## 3. Write a trading strategy

NautilusTrader includes many built-in indicators. In this example we use the MACD indicator to build a simple trading strategy.

You can read more about [MACD here](https://www.investopedia.com/terms/m/macd.asp); this indicator merely serves as an example without any expected alpha. You can also register indicators to receive certain data types; however, in this example we manually pass the received `QuoteTick` to the indicator in the `on_quote_tick` method.


In [19]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.events import PositionClosed
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000


class MACDStrategy(Strategy):
    """A MACD-based strategy that only trades on zero-line crossovers."""

    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        self.trade_size = Quantity.from_int(config.trade_size)

        # Track our position and MACD state
        self.position: Position | None = None
        self.last_macd_above_zero = None  # Track if MACD was above zero on last check

    def on_start(self):
        """Subscribe to market data on strategy start."""
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_stop(self):
        """Clean up on strategy stop."""
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        """Process incoming quote ticks."""
        # Update indicator
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # Check for trading opportunities
        self.check_signals()

    def on_event(self, event: Event):
        """Handle position events."""
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)
            self._log.info(f"Position opened: {self.position.side} @ {self.position.avg_px_open}")
        elif isinstance(event, PositionClosed):
            if self.position and self.position.id == event.position_id:
                self._log.info(f"Position closed with PnL: {self.position.realized_pnl}")
                self.position = None

    def check_signals(self):
        """Check MACD signals - only act on actual crossovers."""
        current_macd = self.macd.value
        current_above_zero = current_macd > 0

        # Skip if this is the first reading
        if self.last_macd_above_zero is None:
            self.last_macd_above_zero = current_above_zero
            return

        # Only act on actual crossovers
        if self.last_macd_above_zero != current_above_zero:
            if current_above_zero:  # Just crossed above zero
                # Only go long if we're not already long
                if not self.is_long:
                    # Close any short position first
                    if self.is_short:
                        self.close_position(self.position)
                    # Then go long (but only when flat)
                    self.go_long()

            else:  # Just crossed below zero
                # Only go short if we're not already short
                if not self.is_short:
                    # Close any long position first
                    if self.is_long:
                        self.close_position(self.position)
                    # Then go short (but only when flat)
                    self.go_short()

        self.last_macd_above_zero = current_above_zero

    def go_long(self):
        """Enter long position only if flat."""
        if self.is_flat:
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
            self._log.info(f"Going LONG - MACD crossed above zero: {self.macd.value:.6f}")

    def go_short(self):
        """Enter short position only if flat."""
        if self.is_flat:
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)
            self._log.info(f"Going SHORT - MACD crossed below zero: {self.macd.value:.6f}")

    @property
    def is_flat(self) -> bool:
        """Check if we have no position."""
        return self.position is None

    @property
    def is_long(self) -> bool:
        """Check if we have a long position."""
        return self.position and self.position.side == PositionSide.LONG

    @property
    def is_short(self) -> bool:
        """Check if we have a short position."""
        return self.position and self.position.side == PositionSide.SHORT

    def on_dispose(self):
        """Clean up on strategy disposal."""

### Enhanced Strategy with Stop-Loss and Take-Profit

The basic MACD strategy above will now generate trades. For better risk management, here's an enhanced version with stop-loss and take-profit orders:

In [20]:
from nautilus_trader.model.objects import Price


class MACDEnhancedConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00005
    exit_threshold: float = 0.00002
    stop_loss_pips: int = 20  # Stop loss in pips
    take_profit_pips: int = 40  # Take profit in pips


class MACDEnhancedStrategy(Strategy):
    """Enhanced MACD strategy with stop-loss and take-profit."""

    def __init__(self, config: MACDEnhancedConfig):
        super().__init__(config=config)
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        self.trade_size = Quantity.from_int(config.trade_size)
        self.position: Position | None = None
        self.last_macd_sign = 0

    def on_start(self):
        """Subscribe to market data on strategy start."""
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_stop(self):
        """Clean up on strategy stop."""
        self.cancel_all_orders(self.config.instrument_id)
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        """Process incoming quote ticks."""
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return

        self.check_signals(tick)

    def on_event(self, event: Event):
        """Handle position events."""
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)
            self._log.info(f"Position opened: {self.position.side} @ {self.position.avg_px_open}")
            # Place stop-loss and take-profit orders
            self.place_exit_orders()
        elif isinstance(event, PositionClosed):
            if self.position and self.position.id == event.position_id:
                pnl = self.position.realized_pnl
                self._log.info(f"Position closed with PnL: {pnl}")
                self.position = None
                # Cancel any remaining exit orders
                self.cancel_all_orders(self.config.instrument_id)

    def check_signals(self, tick: QuoteTick):
        """Check MACD signals and manage positions."""
        current_macd = self.macd.value
        current_sign = 1 if current_macd > 0 else -1

        # Skip if we already have a position
        if self.position:
            return

        # Detect MACD zero-line crossover
        if self.last_macd_sign != 0 and self.last_macd_sign != current_sign:
            if current_sign > 0:
                self.go_long(tick)
            else:
                self.go_short(tick)

        # Entry signals based on threshold
        elif abs(current_macd) > self.config.entry_threshold:
            if current_macd > self.config.entry_threshold:
                self.go_long(tick)
            elif current_macd < -self.config.entry_threshold:
                self.go_short(tick)

        self.last_macd_sign = current_sign

    def go_long(self, tick: QuoteTick):
        """Enter long position."""
        if self.position:
            return  # Already have a position

        order = self.order_factory.market(
            instrument_id=self.config.instrument_id,
            order_side=OrderSide.BUY,
            quantity=self.trade_size,
        )
        self.submit_order(order)
        self._log.info(f"Going LONG @ {tick.ask_price} - MACD: {self.macd.value:.6f}")

    def go_short(self, tick: QuoteTick):
        """Enter short position."""
        if self.position:
            return  # Already have a position

        order = self.order_factory.market(
            instrument_id=self.config.instrument_id,
            order_side=OrderSide.SELL,
            quantity=self.trade_size,
        )
        self.submit_order(order)
        self._log.info(f"Going SHORT @ {tick.bid_price} - MACD: {self.macd.value:.6f}")

    def place_exit_orders(self):
        """Place stop-loss and take-profit orders for the current position."""
        if not self.position:
            return

        entry_price = float(self.position.avg_px_open)
        pip_value = 0.0001  # For FX pairs (adjust for different instruments)

        if self.position.side == PositionSide.LONG:
            # Long position: stop below entry, target above
            stop_price = entry_price - (self.config.stop_loss_pips * pip_value)
            target_price = entry_price + (self.config.take_profit_pips * pip_value)

            # Stop-loss order
            stop_loss = self.order_factory.stop_market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
                trigger_price=Price.from_str(f"{stop_price:.5f}"),
            )
            self.submit_order(stop_loss)

            # Take-profit order
            take_profit = self.order_factory.limit(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
                price=Price.from_str(f"{target_price:.5f}"),
            )
            self.submit_order(take_profit)

            self._log.info(f"Placed LONG exit orders - Stop: {stop_price:.5f}, Target: {target_price:.5f}")

        else:  # SHORT position
            # Short position: stop above entry, target below
            stop_price = entry_price + (self.config.stop_loss_pips * pip_value)
            target_price = entry_price - (self.config.take_profit_pips * pip_value)

            # Stop-loss order
            stop_loss = self.order_factory.stop_market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
                trigger_price=Price.from_str(f"{stop_price:.5f}"),
            )
            self.submit_order(stop_loss)

            # Take-profit order
            take_profit = self.order_factory.limit(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
                price=Price.from_str(f"{target_price:.5f}"),
            )
            self.submit_order(take_profit)

            self._log.info(f"Placed SHORT exit orders - Stop: {stop_price:.5f}, Target: {target_price:.5f}")

    def on_dispose(self):
        """Clean up on strategy disposal."""

## Configuring backtests

Now that we have a trading strategy and data, we can begin to configure a backtest run. Nautilus uses a `BacktestNode` to orchestrate backtest runs, which requires some setup. This may seem a little complex at first, however this is necessary for the capabilities that Nautilus strives for.

To configure a `BacktestNode`, we first need to create an instance of a `BacktestRunConfig`, configuring the following (minimal) aspects of the backtest:

- `engine`: The engine for the backtest representing our core system, which will also contain our strategies.
- `venues`: The simulated venues (exchanges or brokers) available in the backtest.
- `data`: The input data we would like to perform the backtest on.

There are many more configurable features described later in the docs; for now this will get us up and running.


In [21]:
venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"]
)

## 5. Configure data

We need to know about the instruments that we would like to load data for, we can use the `ParquetDataCatalog` for this.

In [22]:
instruments = catalog.instruments()
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

Next, configure the data for the backtest. Nautilus provides a flexible data-loading system for backtests, but that flexibility requires some configuration.

For each tick type (and instrument), we add a `BacktestDataConfig`. In this instance we are simply adding the `QuoteTick`(s) for our EUR/USD instrument:


In [42]:
from nautilus_trader.model import QuoteTick


data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2025-01-10",
)

## 6. Configure engine

Create a `BacktestEngineConfig` to represent the configuration of our core trading system.
Pass in your trading strategies, adjust the log level as needed, and configure any other components (the defaults are fine too).

Add strategies via the `ImportableStrategyConfig`, which enables importing strategies from arbitrary files or user packages. In this instance our `MACDStrategy` lives in the current module, which Python refers to as `__main__`.


In [43]:
# NautilusTrader currently exceeds the rate limit for Jupyter notebook logging (stdout output),
# this is why the `log_level` is set to "ERROR". If you lower this level to see
# more logging then the notebook will hang during cell execution. A fix is currently
# being investigated which involves either raising the configured rate limits for
# Jupyter, or throttling the log flushing from Nautilus.
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDEnhancedStrategy",
            config_path="__main__:MACDEnhancedConfig",
            config={
              "instrument_id": instruments[0].id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)

## 7. Run backtest

We can now pass our various config pieces to the `BacktestRunConfig`. This object now contains the 
full configuration for our backtest.

In [44]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)

The `BacktestNode` class orchestrates the backtest run. This separation between configuration and execution enables the `BacktestNode` to run multiple configurations (different parameters or batches of data). We are now ready to run some backtests.


### Expected Output

When you run the backtest with the improved MACD strategy, you should see:
- **Actual trades being executed** (both BUY and SELL orders).
- **Positions being opened and closed** with proper exit logic.
- **P&L calculations** showing wins and losses.
- **Performance metrics** including win rate, profit factor, and additional statistics.

If you're not seeing any trades, check:
1. The data time range (you may need more data).
2. The threshold parameters (they might be too restrictive).
3. The indicator warm-up period (MACD needs time to initialize).


In [45]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

 # Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

## 8. Analyze results

Now that the run is complete, we can also directly query for the `BacktestEngine`(s) used internally by the `BacktestNode`
by using the run configs ID. 

The engine(s) can provide additional reports and information.

In [47]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

len(engine.trader.generate_order_fills_report())

42

In [48]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,...,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl,is_snapshot
position_id,,,,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-MACDEnhancedStrategy-000-8fd767ac-dcd0-4952-acc2-804b3e21fc59,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-170334-001-000-1,O-20200101-170334-001-000-3,SELL,FLAT,0,1000000,...,2020-01-01 17:03:34.696000+00:00,1577958191609000000,2020-01-02 09:43:11.609000+00:00,59976913000000,1.12120,1.11720,[44.76 USD],0.00357,3955.24 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-87140014-c6a3-4be4-a57e-2f1788d49c81,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200102-094342-001-000-4,O-20200102-094342-001-000-5,BUY,FLAT,0,1000000,...,2020-01-02 09:43:42.539000+00:00,1578019981120000000,2020-01-03 02:53:01.120000+00:00,61758581000000,1.11721,1.11521,[44.64 USD],-0.00179,-2044.64 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-3d44f60d-3168-4aa7-aaea-e6d0e6725e53,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-025301-001-000-7,O-20200103-025301-001-000-8,SELL,FLAT,0,1000000,...,2020-01-03 02:53:01.120000+00:00,1578050438039000000,2020-01-03 11:20:38.039000+00:00,30456919000000,1.11521,1.11721,[44.64 USD],-0.00179,-2044.64 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-7c5ba0fe-60ac-4469-a7c1-af4f16fbb6e5,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-112038-001-000-10,O-20200103-112038-001-000-11,BUY,FLAT,0,1000000,...,2020-01-03 11:20:38.039000+00:00,1578389762174000000,2020-01-07 09:36:02.174000+00:00,339324135000000,1.11721,1.11521,[44.64 USD],-0.00179,-2044.64 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-6eb653f9-35a0-4e56-825f-c717a6926374,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200107-093602-001-000-13,O-20200107-093602-001-000-15,SELL,FLAT,0,1000000,...,2020-01-07 09:36:02.174000+00:00,1578471301092000000,2020-01-08 08:15:01.092000+00:00,81538918000000,1.11521,1.11121,[44.52 USD],0.00359,3955.48 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-1520a7e8-e2af-4b80-befd-729bc521fde4,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-081501-001-000-16,O-20200108-081501-001-000-17,SELL,FLAT,0,1000000,...,2020-01-08 08:15:01.092000+00:00,1578876397999000000,2020-01-13 00:46:37.999000+00:00,405096907000000,1.11116,1.11316,[44.48 USD],-0.00180,-2044.48 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-4f565924-6175-4c5f-9e2c-5f43fd5abbf3,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200113-004637-001-000-19,O-20200113-004637-001-000-20,BUY,FLAT,0,1000000,...,2020-01-13 00:46:37.999000+00:00,1578989731145000000,2020-01-14 08:15:31.145000+00:00,113333146000000,1.11316,1.11116,[44.48 USD],-0.00180,-2044.48 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-71b86a4e-9fbd-452f-b19d-179a4526ee5e,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200114-081531-001-000-22,O-20200114-081531-001-000-23,SELL,FLAT,0,1000000,...,2020-01-14 08:15:31.145000+00:00,1579004046425000000,2020-01-14 12:14:06.425000+00:00,14315280000000,1.11116,1.11316,[44.48 USD],-0.00180,-2044.48 USD,True
EUR/USD.SIM-MACDEnhancedStrategy-000-33b7ede8-563e-491b-806b-b562e0d297ab,BACKTESTER-001,MACDEnhancedStrategy-000,EUR/USD.SIM,SIM-001,O-20200114-121406-001-000-25,O-20200114-121406-001-000-27,BUY,FLAT,0,1000000,...,2020-01-14 12:14:06.425000+00:00,1579159959475000000,2020-01-16 07:32:39.475000+00:00,155913050000000,1.11316,1.11716,[44.60 USD],0.00359,3955.40 USD,True


In [49]:
engine.trader.generate_account_report(Venue("SIM"))

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,[],True,{}
2020-01-01 17:03:34.696000+00:00,999977.58,33636.00,966341.58,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-01 17:03:34.696000+00:00,999977.58,67332.00,932645.58,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '33696.0...",False,{}
2020-01-01 17:03:34.696000+00:00,999977.58,100848.00,899129.58,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '67212.0...",False,{}
2020-01-02 09:43:11.609000+00:00,1003955.24,33696.00,970259.24,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '33696.0...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2020-01-31 16:26:55.578000+00:00,1007089.15,66617.40,940471.75,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '33338.7...",False,{}
2020-01-31 16:26:55.578000+00:00,1007089.15,99776.10,907313.05,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '66497.4...",False,{}
2020-01-31 16:59:56.198000+00:00,1007089.15,66437.40,940651.75,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '33158.7...",False,{}
2020-01-31 16:59:56.198000+00:00,1007089.15,33278.70,973810.45,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}


## 9. Performance Metrics

Let's add some additional performance metrics to better understand how our strategy performed:

In [50]:
# Get performance statistics

# Get the account and positions
account = engine.trader.generate_account_report(Venue("SIM"))
positions = engine.trader.generate_positions_report()
orders = engine.trader.generate_order_fills_report()

# Print summary statistics
print("=== STRATEGY PERFORMANCE ===")
print(f"Total Orders: {len(orders)}")
print(f"Total Positions: {len(positions)}")

if len(positions) > 0:
    # Convert P&L strings to numeric values
    positions["pnl_numeric"] = positions["realized_pnl"].apply(
        lambda x: float(str(x).replace(" USD", "").replace(",", "")) if isinstance(x, str) else float(x)
    )

    # Calculate win rate
    winning_trades = positions[positions["pnl_numeric"] > 0]
    losing_trades = positions[positions["pnl_numeric"] < 0]

    win_rate = len(winning_trades) / len(positions) * 100 if len(positions) > 0 else 0

    print(f"\nWin Rate: {win_rate:.1f}%")
    print(f"Winning Trades: {len(winning_trades)}")
    print(f"Losing Trades: {len(losing_trades)}")

    # Calculate returns
    total_pnl = positions["pnl_numeric"].sum()
    avg_pnl = positions["pnl_numeric"].mean()
    max_win = positions["pnl_numeric"].max()
    max_loss = positions["pnl_numeric"].min()

    print(f"\nTotal P&L: {total_pnl:.2f} USD")
    print(f"Average P&L: {avg_pnl:.2f} USD")
    print(f"Best Trade: {max_win:.2f} USD")
    print(f"Worst Trade: {max_loss:.2f} USD")

    # Calculate risk metrics if we have both wins and losses
    if len(winning_trades) > 0 and len(losing_trades) > 0:
        avg_win = winning_trades["pnl_numeric"].mean()
        avg_loss = abs(losing_trades["pnl_numeric"].mean())
        profit_factor = winning_trades["pnl_numeric"].sum() / abs(losing_trades["pnl_numeric"].sum())

        print(f"\nAverage Win: {avg_win:.2f} USD")
        print(f"Average Loss: {avg_loss:.2f} USD")
        print(f"Profit Factor: {profit_factor:.2f}")
        print(f"Risk/Reward Ratio: {avg_win/avg_loss:.2f}")
else:
    print("\nNo positions generated. Check strategy parameters.")

print("\n=== FINAL ACCOUNT STATE ===")
print(account.tail(1).to_string())

=== STRATEGY PERFORMANCE ===
Total Orders: 42
Total Positions: 21

Win Rate: 38.1%
Winning Trades: 8
Losing Trades: 13

Total P&L: 6836.96 USD
Average P&L: 325.57 USD
Best Trade: 3955.94 USD
Worst Trade: -2044.64 USD

Average Win: 3955.61 USD
Average Loss: 1908.30 USD
Profit Factor: 1.28
Risk/Reward Ratio: 2.07

=== FINAL ACCOUNT STATE ===
                                       total locked        free currency account_id account_type base_currency margins  reported info
2020-01-31 16:59:56.198000+00:00  1006836.96   0.00  1006836.96      USD    SIM-001       MARGIN           USD      []     False   {}
